# 검색인덱스

In [ ]:
import meilisearch
import json

# meilisearch의 클라이언트의 객체
# client = meilisearch.Client('로컬호스트', '서버의키')
client = meilisearch.Client('http://localhost:7700', 'aSampleMasterKey')

# json_file = open('data/movies.json', encoding='utf-8')
# movies = json.load(json_file)
# client.index('movies').add_documents(movies)

In [6]:
import pandas as pd
df = pd.read_csv('./data/nasdaq_screener_1749314966785.csv', na_filter=False)  
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$115.80,1.8600,1.632%,32894695502.00,United States,1999,1556432,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$28.01,0.1400,0.502%,7251854487.00,United States,2016,3699572,Industrials,Aluminum
2,AACB,Artius II Acquisition Inc. Class A Ordinary Sh...,$10.11,0.0000,0.00%,0.00,United States,2025,11698,,
3,AACBR,Artius II Acquisition Inc. Rights,$0.24,0.0000,0.00%,0.00,United States,2025,16732,,
4,AACBU,Artius II Acquisition Inc. Units,$10.32,-0.0199,-0.192%,0.00,United States,2025,10156,Finance,Blank Checks


# meilisearch 사용 방법

- Vector 서치 지원, 클라우드 버전도 지원함(유료)

## 전처리 하기

In [10]:
# 특수문자가 포함된 값만 필터링 (정규표현식 사용)
df[df['Symbol'].str.contains(r'[^a-zA-Z0-9]', regex=True)]


,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
38,ABR^D,Arbor Realty Trust 6.375% Series D Cumulative ...,$17.0463,-0.0046,-0.027%,,United States,,8028,,
39,ABR^E,Arbor Realty Trust 6.25% Series E Cumulative R...,$16.85,-0.0650,-0.384%,,United States,,5605,,
40,ABR^F,Arbor Realty Trust 6.25% Series F Fixed-to-Flo...,$20.38,0.0200,0.098%,,United States,,7502,,
79,ACP^A,abrdn Income Credit Strategies Fund 5.250% Ser...,$20.80,0.0000,0.00%,,United States,,342,,
81,ACR^C,ACRES Commercial Realty Corp. 8.625% Fixed-to-...,$25.10,0.0490,0.196%,,United States,,1698,,
...,...,...,...,...,...,...,...,...,...,...,...
6769,WRB^F,W.R. Berkley Corporation 5.10% Subordinated De...,$19.59,-0.0600,-0.305%,,United States,,23288,,
6770,WRB^G,W.R. Berkley Corporation 4.25% Subordinated De...,$16.28,-0.0800,-0.489%,,United States,,15520,,
6771,WRB^H,W.R. Berkley Corporation 4.125% Subordinated D...,$15.93,-0.0700,-0.438%,,United States,,23443,,
6785,WSO/B,Watsco Inc.,$450.00,0.0000,0.00%,,United States,,21,,


In [11]:
# r'[/^]'는 정규 표현식(Regex)으로, 문자열 내에서 슬래시(/)나 캐럿(^) 문자를 찾아서 이를 밑줄(_)로 대체함.
df['id'] =df['Symbol'].str.strip().replace(r'[/^]', '_', regex=True)

In [12]:
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,id
0,A,Agilent Technologies Inc. Common Stock,$115.80,1.8600,1.632%,32894695502.00,United States,1999,1556432,Industrials,Biotechnology: Laboratory Analytical Instruments,A
1,AA,Alcoa Corporation Common Stock,$28.01,0.1400,0.502%,7251854487.00,United States,2016,3699572,Industrials,Aluminum,AA
2,AACB,Artius II Acquisition Inc. Class A Ordinary Sh...,$10.11,0.0000,0.00%,0.00,United States,2025,11698,,,AACB
3,AACBR,Artius II Acquisition Inc. Rights,$0.24,0.0000,0.00%,0.00,United States,2025,16732,,,AACBR
4,AACBU,Artius II Acquisition Inc. Units,$10.32,-0.0199,-0.192%,0.00,United States,2025,10156,Finance,Blank Checks,AACBU


# 데이터 인덱싱 하기

In [ ]:
# index 생성 (nasdaq 라는 이름으로),  반드시 dict로 변경해야함. 서치 엔진에 업로딩하기
client.index('nasdaq').add_documents(
    df.to_dict(orient="records"), 
    primary_key='id'
)

TaskInfo(task_uid=2, index_uid='nasdaq', status='enqueued', type='documentAdditionOrUpdate', enqueued_at=datetime.datetime(2025, 6, 7, 17, 25, 14, 654956))

In [18]:
# 서치하기
# client.index('nasdaq').search("Apple")
client.index('nasdaq').search("AAPL")

{'hits': [{'id': 'AAPL',
   'Symbol': 'AAPL',
   'Name': 'Apple Inc. Common Stock',
   'Last Sale': '$203.92',
   'Net Change': 3.29,
   '% Change': '1.64%',
   'Market Cap': '3045713637920.00',
   'Country': 'United States',
   'IPO Year': '1980',
   'Volume': 46538043,
   'Sector': 'Technology',
   'Industry': 'Computer Manufacturing'}],
 'query': 'AAPL',
 'processingTimeMs': 0,
 'limit': 20,
 'offset': 0,
 'estimatedTotalHits': 1}

# 인덱스 삭제

In [15]:
# 인덱스 삭제
client.delete_index('nasdaq')

TaskInfo(task_uid=1, index_uid='nasdaq', status='enqueued', type='indexDeletion', enqueued_at=datetime.datetime(2025, 6, 7, 17, 24, 52, 599123))